In [1]:
# ================================
# 1. 라이브러리 & 시드 설정
# ================================
import os
import random
import numpy as np
from PIL import Image # ★ PIL 유지 ★

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm import tqdm
import copy

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report
)

# ⭐️ 클래스 개수 정의 ⭐️
NUM_CLASSES = 4
TARGET_CLASSES = ["anger", "happy", "panic", "sadness"] # ImageFolder 기본 인덱스 순서를 가정

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


# ================================
# 2. Dataset 정의 (폴더 기반 4진 분류로 변경)
# ================================
class MulticlassImageDataset(Dataset): # ⭐️ 클래스 이름 변경 ⭐️
    """주어진 폴더 구조에서 4가지 감정 클래스를 로드합니다."""
    def __init__(self, data_root, split="train", transform=None):
        """
        data_root: /workspace/merge_data (루트 경로)
        split: 'train' | 'val'
        """
        self.transform = transform
        self.samples = []
        
        # 'train' 폴더: data_root/train (또는 new_data/img_multiclass 등으로 경로 변경 필요)
        # ⭐️ 새로운 경로 구조를 가정: data_root/split/emotion_folders ⭐️
        img_base_dir = os.path.join(data_root,"data","img", split) 
        
        # ⭐️ 4진 감정 폴더 목록 (인덱스 순서가 중요합니다) ⭐️
        self.classes = {name: i for i, name in enumerate(TARGET_CLASSES)} # anger: 0, happy: 1, panic: 2, sadness: 3

        print(f"--- {split} 데이터 로드 시작: {img_base_dir} ---")
        
        for class_name, label in self.classes.items():
            class_dir = os.path.join(img_base_dir, class_name)
            
            if not os.path.isdir(class_dir):
                print(f"[WARNING] 폴더를 찾을 수 없습니다. 경로 확인 필요: {class_dir}")
                continue

            for filename in os.listdir(class_dir):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
                    img_path = os.path.join(class_dir, filename)
                    self.samples.append((img_path, label))

        # 클래스별 카운트 확인
        class_counts = {name: 0 for name in TARGET_CLASSES}
        for _, label in self.samples:
            class_counts[TARGET_CLASSES[label]] += 1

        print(f"✅ {split} samples: {len(self.samples)}")
        print(f"   Class Counts: {class_counts}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        try:
            # ★ PIL로 로드 (BGR -> RGB 변환 불필요) ★
            image = Image.open(img_path).convert("RGB") 
        except Exception as e:
            print(f"이미지 로드 실패: {img_path}, {e}")
            return None, None 

        if self.transform is not None:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.long)
        return image, label


# ================================
# 3. Transform / Dataset / DataLoader
# ================================
# ResNet 기본 입력 크기 224x224 기준
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# ★★★ 경로 수정 필요: 4진 분류 데이터셋의 루트 폴더 경로 ★★★
# 경료:merge_data
IMG_BASE_ROOT = "/workspace/merge_data" 


train_dataset = MulticlassImageDataset(IMG_BASE_ROOT, split="train", transform=train_transform) # ⭐️ Dataset 변경 ⭐️
val_dataset   = MulticlassImageDataset(IMG_BASE_ROOT, split="val",   transform=val_transform) # ⭐️ Dataset 변경 ⭐️

batch_size = 32
num_workers = 4

# DataLoader 생성 시, None 값을 필터링하기 위해 collate_fn을 정의합니다. (유지)
def collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    if not batch:
        return None, None 
    return torch.utils.data.dataloader.default_collate(batch)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, collate_fn=collate_fn)
val_loader   = DataLoader(val_dataset,   batch_size=batch_size, shuffle=False,
                          num_workers=num_workers, collate_fn=collate_fn)


# ================================
# 4. 모델 정의 (ResNet18, 4-class) - ⭐️ 출력 클래스 변경 ⭐️
# ================================
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, NUM_CLASSES) # ⭐️ 2 -> NUM_CLASSES(4)로 변경 ⭐️
model = model.to(device)

# 가중치 없는 기본 CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)


# ================================
# 5. 학습 함수 (변경 없음)
# ================================
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 30)

        # ---- Train ----
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            if inputs is None: continue # collate_fn에서 None 반환 시 스킵

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)
        
        # total이 0인 경우를 방지
        if total == 0:
            print("Warning: No valid training samples found in this epoch.")
            continue 

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total
        print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.inference_mode():
            for inputs, labels in val_loader:
                if inputs is None: continue # collate_fn에서 None 반환 시 스킵

                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)
                val_total += labels.size(0)

        if val_total == 0:
            print("Warning: No valid validation samples found.")
            val_acc = 0.0
        else:
            val_loss /= val_total
            val_acc = val_corrects.double() / val_total
            print(f"Val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

        # best model 저장
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            # ⭐️ 저장 파일명 변경 ⭐️
            torch.save(best_model_wts, "best_resnet_4_emotions.pth") 
            print(">> best model updated")

    print(f"\nBest val Acc: {best_val_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model


# ================================
# 6. 학습 실행
# ================================
num_epochs = 10
# model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs) # 다음 블록에서 실행 권장
print("\n\n=== 6. 학습 시작 ===")
model = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs)


# ================================
# 7. 성능 지표 (val셋을 평가용으로 사용)
# ================================
model.eval()
all_labels = []
all_preds = []

with torch.inference_mode():
    for inputs, labels in val_loader:
        if inputs is None: continue
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# 예측 결과가 있을 경우에만 평가 지표 계산
if len(all_labels) > 0:
    acc  = accuracy_score(all_labels, all_preds)
    # ⭐️ 다중 클래스 분류에 맞게 average='weighted' 사용 및 pos_label 제거 ⭐️
    prec = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    rec  = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1   = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    cm   = confusion_matrix(all_labels, all_preds)

    print("\n=== Val Metrics (4-class: anger, happy, panic, sadness) ===")
    print(f"Accuracy : {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall   : {rec:.4f}")
    print(f"F1-score : {f1:.4f}")
    print("\nConfusion Matrix (rows: true, cols: pred)")
    print(cm)

    print("\nClassification report:")
    # ⭐️ target_names를 4개 클래스로 지정 ⭐️
    print(classification_report(all_labels, all_preds, target_names=TARGET_CLASSES, zero_division=0))
else:
    print("\nWarning: No valid samples found for evaluation.")

device: cuda
--- train 데이터 로드 시작: /workspace/merge_data/data/img/train ---
✅ train samples: 5841
   Class Counts: {'anger': 1424, 'happy': 1493, 'panic': 1453, 'sadness': 1471}
--- val 데이터 로드 시작: /workspace/merge_data/data/img/val ---
✅ val samples: 1151
   Class Counts: {'anger': 291, 'happy': 299, 'panic': 280, 'sadness': 281}


=== 6. 학습 시작 ===

Epoch 1/10
------------------------------


100%|██████████| 183/183 [02:14<00:00,  1.36it/s]

Train Loss: 0.8759 Acc: 0.6287


Val   Loss: 0.6632 Acc: 0.7576
>> best model updated

Epoch 2/10
------------------------------


100%|██████████| 183/183 [02:13<00:00,  1.37it/s]

...

Train Loss: 0.0570 Acc: 0.9800


Val   Loss: 0.9441 Acc: 0.7602
>> best model updated

Epoch 10/10
------------------------------


100%|██████████| 183/183 [02:14<00:00,  1.36it/s]

Train Loss: 0.0370 Acc: 0.9889


Val   Loss: 1.0097 Acc: 0.7333

Best val Acc: 0.7602

=== Val Metrics (4-class: anger, happy, panic, sadness) ===
Accuracy : 0.7602
Precision: 0.7619
Recall   : 0.7602
F1-score : 0.7591

Confusion Matrix (rows: true, cols: pred)
[[196  12  49  34]
 [ 10 268  17   4]
 [ 34  21 216   9]
 [ 37  17  32 195]]

Classification report:
              precision    recall  f1-score   support

       anger       0.71      0.67      0.69       291
       happy       0.84      0.90      0.87       299
       panic       0.69      0.77      0.73       280
     sadness       0.81      0.69      0.75       281

    accuracy                           0.76      1151
   macro avg       0.76      0.76      0.76      1151
weighted avg       0.76      0.76      0.76      1151

